In [ ]:
import laspy
import numpy as np
import pandas as pd
import os

In [ ]:
#the folder needs to have a luminance and a non-luminance csv file from the ljx software, this lets you build an ljx processor

folder = r"C:\Users\eashenhurst\Desktop\local scans\test_csv_core\test 1"

csv_files = [f for f in os.listdir(folder) if f.endswith(".csv")]

for csv_file in csv_files:
    is_luminance = "luminance" in csv_file.lower()
    file_path = os.path.join(folder, csv_file)

    df = pd.read_csv(file_path, header=None)
    z = df.to_numpy().flatten().tolist()

    x_axis_size = 0.165
    y_axis_size = 0.225

    x_values = np.arange(0, 10000) * x_axis_size
    y_values = np.arange(-1599.5, 1600, 1) * y_axis_size

    x = np.repeat(x_values, 3200)
    y = np.tile(y_values, 10000)

    xyz = np.column_stack((x, y, z)).astype(np.float32)

    base_name = "intensity_cloud" if is_luminance else "point_cloud"

    bpc_path = os.path.join(folder, base_name + ".bpc")
    xyz.tofile(bpc_path)

    las_path = os.path.join(folder, base_name + ".las")
    header = laspy.LasHeader(point_format=1, version="1.2")
    header.scale = [0.0001, 0.0001, 0.001] 
    las = laspy.LasData(header)
    las.x = np.array(x, dtype = np.float32)
    las.y = np.array(y, dtype = np.float32) 
    las.z = np.array(z, dtype = np.float32)
    las.write(las_path)

    print(f"Processed: {csv_file} → {base_name}.bpc/.las")

In [ ]:
bpc_path = r"C:\Users\eashenhurst\Downloads\LIDAR_001.bpc"

point_cloud = np.fromfile(bpc_path, dtype=np.float32).reshape(-1, 3)

print(point_cloud.shape)

print(point_cloud[0])

nanmask = np.where(np.isnan(point_cloud[:, 2]))

point_cloud[nanmask,2] = 0
x = point_cloud[:,0]
y = point_cloud[:,1]
z = point_cloud[:,2]

las_path = os.path.join(r"C:\Users\eashenhurst\Desktop", "point_cloud" + ".las")
header = laspy.LasHeader(point_format=1, version="1.2")
header.scale = [0.0001, 0.0001, 0.001] 
las = laspy.LasData(header)
las.x = np.array(x)
las.y = np.array(y)
las.z = np.array(z)
las.write(las_path)

